In [1]:
import random
import numpy as np
from data_process import get_CIFAR10_data
%matplotlib inline
from save_submission import output_submission_csv

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [2]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [3]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [4]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

# Get Accuracy
This function computes how well your model performs using accuracy as a metric.


In [5]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.


The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


# Model Perceptron

In [6]:
import numpy as np


class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.w = None ###### TODO: change this
        self.lr = lr ###### TODO: change this
        self.epochs = n_epochs ###### TODO: change this
        self.n_class = n_class_CIFAR ###### TODO: change this

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the perceptron update rule as introduced in Lecture 3.
        Parameters:
            X_train: a number array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        self.w = np.random.randn(self.n_class, D)

        ###### YOUR CODE STARTS HERE ######
        
        for epoch in range(self.epochs):
            for i in range(N):
                y_pred = np.argmax(np.dot(self.w,X_train[i].T))
        
                if (y_train[i]!=y_pred ):
                    self.w[y_train[i]] += self.lr*X_train[i]
                    self.w[y_pred] -= self.lr*X_train[i]
            
        ###### YOUR CODE ENDS HERE ######
        pass

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ###### YOUR CODE STARTS HERE ######
        
        for i in range(N):
            y_test_pred = np.argmax(np.dot(self.w,X_test[i].T))
            y_test[i] = y_test_pred
        
        ###### YOUR CODE ENDS HERE ######
        return y_test

## Train Perceptron on CIFAR

In [7]:
lr = 0.001
n_epochs = 30

percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [8]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 31.567500


### Validate Perceptron on CIFAR

In [9]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 26.250000


### Test Perceptron on CIFAR

In [10]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 26.340000


In [11]:
output_submission_csv('output/Logistic_submission_CIFAR.csv', percept_CIFAR.predict(X_test_CIFAR))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.




The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [12]:
"""Logistic regression model."""

import numpy as np
import math
from scipy.special import expit


class Logistic:
    def __init__(self, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.w = None###### TODO: change this
        self.lr = learning_rate###### TODO: change this
        self.epochs = n_epochs###### TODO: change this
        #self.threshold = ###### TODO: change this

    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        """Sigmoid function.
        Parameters:
            z: the input
        Returns:
            the sigmoid of the input
        """
        ##### YOUR CODE STARTS HERE #####
        
        return expit(z)
        
        ##### YOUR CODE ENDS HERE #####
        return ret

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the logistic regression update rule as introduced in lecture.
        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        self.w = np.random.randn(1, D)

        ##### YOUR CODE STARTS HERE #####
        for i in range(self.epochs):
            y_predict = self.sigmoid(np.dot(self.w,X_train.T))
            dw = (1/N) * (np.dot((y_predict-y_train),X_train))
            self.w -= self.lr * dw
        
        ##### YOUR CODE ENDS HERE #####
        pass

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ##### YOUR CODE STARTS HERE #####
        
        y_predict = self.sigmoid(np.dot(self.w,X_test.T))
        y_test = np.where(y_predict > 0.5, 1,0)
        
        ##### YOUR CODE ENDS HERE #####
        return y_test

### Training Logistic Classifer

In [13]:
learning_rate = 0.001
n_epochs = 100

lr = Logistic(learning_rate, n_epochs)
lr.train(X_train_CIFAR, y_train_CIFAR)

In [14]:
pred_lr = lr.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_CIFAR)))

The training accuracy is given by: 12.200000


### Validate Logistic Classifer

In [15]:
pred_lr = lr.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_CIFAR)))

The validation accuracy is given by: 12.150000


### Test Logistic Classifier

In [16]:
pred_lr = lr.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_CIFAR)))
output_submission_csv('output/Logistic_submission_CIFAR.csv', lr.predict(X_test_CIFAR))

The testing accuracy is given by: 12.390000
